# Imports

In [170]:
from ultralytics import YOLO
import os
from azure.storage.blob import BlobServiceClient
from pydicom import dcmread
from io import BytesIO
import numpy as np
import cv2
import pylidc as pl

# Helper Functions

## Return ann from slice

In [171]:
# Return annotations
def get_ann_from_slice(patient_id, slice_location):
    # Fetch the scan for the patient
    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == patient_id).first()
    if not scan:
        raise ValueError(f"No scan found for patient ID {patient_id}")

    # Initialize list to store annotations
    annotations = []

    # Iterate over all annotations in the scan
    for ann in scan.annotations:
        for contour in ann.contours:
            # Check if the annotation is close to the specified slice location
            if abs(contour.image_z_position - slice_location) < scan.slice_spacing:
                annotations.append(ann)
                break

    return annotations

## Returns if model result is positive

In [172]:
def isNodule(result):
    return int(len(result.boxes.xywh.tolist()) > 0)

# Generate Results

In [173]:
model = YOLO('/home/andrew/ITRI-LungCancer/YOLO/model.pt')

path = '/home/andrew/ITRI-LungCancer/sample/'
imgs = os.listdir(path)
imgs.sort()

model_input = []
annotations = []
for i, img in enumerate(imgs):
    patient_id = img.split('_')[0]
    slice_location = float(img.split('_')[-1].split('.')[0])
    
    annotations.append(get_ann_from_slice(patient_id, slice_location))
    model_input.append(path+img)
results = model(model_input, verbose=False)

# Compare

In [174]:
!rm -rf /home/andrew/ITRI-LungCancer/sample_out
!mkdir -p /home/andrew/ITRI-LungCancer/sample_out

outpath = '/home/andrew/ITRI-LungCancer/sample_out/'

for result, ann, name, in zip(results, annotations, imgs):
    if isNodule(result):
        x_center, y_center, width, height = result.boxes.xywh[0].tolist()

        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        image = result.orig_img
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
        
        if ann != []:
            for a in ann:
                x, y, z = a.centroid
                x = int(x)
                y = int(y)
                cv2.circle(image, (y, x), 15, (0, 0, 255), 1)
        cv2.imwrite(f'{outpath}/{name}.jpg', image)

4701.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
4706.71s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
